In [1]:
import gym

env = gym.make("Taxi-v3", render_mode='human').env
env.reset()
env.render()

env.reset: Resets the environment and returns a random initial state.
env.step(action): Step the environment by one timestep. Returns

- observation: Observations of the environment
- reward: If your action was beneficial or not
- done: Indicates if we have successfully picked up and dropped off a passenger, also called one episode
- info: Additional info such as performance and latency for debugging purposes

There are 4 locations (labeled by different letters), and our job is to pick up the passenger at one location and drop him off at another. We receive +20 points for a successful drop-off and lose 1 point for every time-step it takes. There is also a 10 point penalty for illegal pick-up and drop-off actions.

In [2]:
env.reset() # reset environment to a new, random state
env.render()

print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))


Action Space Discrete(6)
State Space Discrete(500)


Action space is 500 because we have 5x5x5x4 possible states(5x5 positions, 5 person position(4 out the taxi 1 in the taxi) 4 destinations)

Actions are numbered from 0 to f and are south, north, east, west,pickup,dropoff

The optimal action for each state is the action that has the highest cumulative long-term reward.

In [3]:
state = env.encode(3, 1, 2, 0) 
# (taxi row, taxi column, passenger index, destination index)
print("State:", state)

env.s = state
env.render()

State: 328


When the Taxi environment is created, there is an initial Reward table that's also created, called `P`. We can think of it like a matrix that has the number of states as rows and number of actions as columns, i.e. a 
states x actions matrix.

This dictionary has the structure {action: [(probability, nextstate, reward, done)]}

In [4]:
env.P[328]

{0: [(1.0, 428, -1, False)],
 1: [(1.0, 228, -1, False)],
 2: [(1.0, 348, -1, False)],
 3: [(1.0, 328, -1, False)],
 4: [(1.0, 328, -10, False)],
 5: [(1.0, 328, -10, False)]}

## Solving without RL

Since we have our P table for default rewards in each state, we can try to have our taxi navigate just using that.

We'll create an infinite loop which runs until one passenger reaches one destination (one episode), or in other words, when the received reward is 20. The env.action_space.sample() method automatically selects one random action from set of all possible actions

In [5]:
env.step(env.action_space.sample())

(289,
 -1,
 False,
 False,
 {'prob': 1.0, 'action_mask': array([1, 1, 0, 1, 0, 0], dtype=int8)})

In [6]:
env.s = 328
epochs = 0
penalties, reward = 0, 0

frames = [] #for animation
done = False

while not done:
    action = env.action_space.sample()
    state, reward, done, info, _ = env.step(action)
    
    if reward == -10:
        penalties += 1
    
        
    # Put each rendered frame into dict for animation
    frames.append({
        'frame': env.render(),
        'state': state,
        'action': action,
        'reward': reward
        }
    )
    
    epochs += 1
    print(epochs)
    
    
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

1
2
3
4
5
6
7
8


KeyboardInterrupt: 

In [ ]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'].getvalue())
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)
        
print_frames(frames)

Not good. Our agent takes thousands of timesteps and makes lots of wrong drop offs to deliver just one passenger to the right destination.

This is because we aren't learning from past experience. We can run this over and over, and it will never optimize. The agent has no memory of which action was best for each state, which is exactly what Reinforcement Learning will do for us.

## Reinforcement Learning

## Intro to Q-learning
Essentially, Q-learning lets the agent use the environment's rewards to learn, over time, the best action to take in a given state.

In our Taxi environment, we have the reward table, P, that the agent will learn from. It does thing by looking receiving a reward for taking an action in the current state, then updating a Q-value to remember if that action was beneficial.

The values store in the Q-table are called a Q-values, and they map to a (state, action) combination.

A Q-value for a particular state-action combination is representative of the "quality" of an action taken from that state. Better Q-values imply better chances of getting greater rewards.

For example, if the taxi is faced with a state that includes a passenger at its current location, it is highly likely that the Q-value for pickup is higher when compared to other actions, like dropoff or north.

Q-values are initialized to an arbitrary value, and as the agent exposes itself to the environment and receives different rewards by executing different actions, the Q-values are updated using the equation:

We are assigning or updating, the Q-value of the agent's current state and action by first taking a weight of the old Q-value, then adding the learned value. The learned value is a combination of the reward for taking the current action in the current state, and the discounted maximum reward from the next state we will be in once we take the current action.

The Q-table is a matrix where we have a row for every state (500) and a column for every action (6). It's first initialized to 0, and then values are updated after training. Note that the Q-table has the same dimensions as the reward table, but it has a completely different purpose

Breaking it down into steps, we get

- Initialize the Q-table by all zeros.
- Start exploring actions: For each state, select any one among   all possible actions for the current state (S).
- Travel to the next state (S') as a result of that action (a).
- For all possible actions from the state (S') select the one     with the highest Q-value.
- Update Q-table values using the equation.
- Set the next state as the current state.
- If goal state is reached, then end and repeat the process.


We want to prevent the action from always taking the same route, and possibly overfitting, so we'll be introducing another parameter called 
ϵ
 "epsilon" to cater to this during training.

Instead of just selecting the best learned Q-value action, we'll sometimes favor exploring the action space further. Lower epsilon value results in episodes with more penalties (on average) which is obvious because we are exploring and making random decisions.

In [6]:
import torch as t

q_table = t.zeros([env.observation_space.n, env.action_space.n], dtype=t.float32)

In [7]:
%%time

import random
from IPython.display import clear_output

alpha = 0.5
gamma = 0.8
epsilon = 0.1

all_epochs = []
all_penalties = []


for i in range(1,100001):
    state = list(env.reset())
    
    epochs, penalties, reward = 0, 0, 0
    done = False
    
    while not done:
        
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample()# Explore action space
        else:
            action = t.argmax(q_table[state[0]]).item()# Exploit learned values
            
        next_state, reward, done, info, _ = env.step(action)
        
        old_value = q_table[state[0], action]
        next_max = t.max(q_table[next_state])

        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state[0], action] = new_value.item()
        #print(new_value)
        
        if reward == -10:
            penalties += 1
        
        state[0] = next_state
        epochs += 1
        
        print(reward, next_state, action, q_table[state[0], action], done)
        
        
    if i % 10 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")
        
        

-1 244 0 tensor(0.) False
-1 344 0 tensor(0.) False
-1 444 0 tensor(0.) False
-1 444 0 tensor(-0.5000) False
-1 344 1 tensor(0.) False
-1 444 0 tensor(-0.5000) False
-1 444 2 tensor(-0.5000) False
-1 424 3 tensor(0.) False
-1 424 0 tensor(-0.5000) False
-1 324 1 tensor(0.) False
-1 424 0 tensor(-0.5000) False
-1 444 2 tensor(-0.5000) False


KeyboardInterrupt: 

In [7]:
q_table[328]

tensor([0., 0., 0., 0., 0., 0.])

In [8]:
q_table = t.load("q_table.pt")
#t.save(q_table, 'q_table.pt')

Let's evaluate the performance of our agent. We don't need to explore actions any further, so now the next action is always selected using the best Q-value:

In [22]:
total_epochs = 0
total_penalties = 0
episodes = 10


for i in range(episodes):
    state = list(env.reset())
    state[0] = 81 
    
    epochs, penalties, reward = 0, 0, 0
    done = False
    
    while not done:
        
        action = t.argmax(q_table[state[0]]).item()# Exploit learned values
        
        state[0], reward, done, info, _ = env.step(action)
        #print(done)

        if reward == -10:
            penalties += 1
            
        epochs += 1
        
    print(i)    
    total_penalties += penalties
    total_epochs += epochs

print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")
        
        

0
1
2
3
4
5
6
7
8
9
Average timesteps per episode: 13.8
Average penalties per episode: 0.0


The values of `alpha`, `gamma`, and `epsilon` were mostly based on intuition and some "hit and trial", but there are better ways to come up with good values.

Ideally, all three should decrease over time because as the agent continues to learn, it actually builds up more resilient priors;

A simple way to programmatically come up with the best set of values of the hyperparameter is to create a comprehensive search function (similar to grid search) that selects the parameters that would result in best reward/time_steps ratio. The reason for reward/time_steps is that we want to choose parameters which enable us to get the maximum reward as fast as possible.